In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

from loop import callbacks as C
from loop.config import defaults
from loop.metrics import accuracy
from loop.modules import TinyNet
from loop.phase import Phase
from loop.utils import unwrap_if_single

In [ ]:
#export
def default_optimizer(model, **params):
    if 'lr' not in params:
        params['lr'] = 0.001
    return optim.Adam(model.parameters(), **params)

In [ ]:
#export
def create_callbacks(cbs, default: bool=True):
    defaults = [C.RollingLoss(), C.History(), C.StreamLogger()] if default else []
    cbs = list(cbs or [])
    cbs += defaults
    return C.Group(cbs)

In [ ]:
#export
class Loop:
    def __init__(self, model: nn.Module, cbs: list=None,
                 default_cb: bool=True, opt_fn: 'callable'=default_optimizer,
                 opt_params: dict=None, device: 'device'=defaults.device,
                 loss_fn: 'callable'=defaults.loss_fn):
        
        model.to(device)
        opt = opt_fn(model, **(opt_params or {}))
        cb = create_callbacks(cbs, default_cb)
        cb.set_model(model)
        
        self.model = model
        self.opt = opt
        self.cb = cb
        self.loss_fn = loss_fn 
        self.device = device
        
    def fit_datasets(self, trn_ds, val_ds, epochs: int=1, batch_size: int=defaults.bs):
        phases = Phase.make_train_valid(
            trn_ds, val_ds, bs=batch_size, num_workers=defaults.n_jobs)
        self.train(phases, epochs)
        
    def train(self, phases: list, epochs: int=1):
        try:
            self.cb.training_started(phases=phases)
            for epoch in range(1, epochs + 1):
                self.train_one_epoch(phases, epoch)
            self.cb.training_ended(phases=phases)
        except TrainingInterrupted as e:
            self.cb.interrupted(reason=e)
    
    def train_one_epoch(self, phases: list, curr_epoch: int=1):
        cb, model, opt = self.cb, self.model, self.opt
        
        cb.epoch_started(epoch=curr_epoch)

        for phase in phases:
            n = len(phase.loader)
            cb.phase_started(phase=phase, total_batches=n)
            is_training = phase.grad
            model.train(is_training)
            
            for batch in phase.loader:
                phase.batch_index += 1
                cb.batch_started(phase=phase, total_batches=n)
                x, y = place_and_unwrap(batch, self.device)
                
                with torch.set_grad_enabled(is_training):
                    cb.before_forward()
                    out = model(x)
                    cb.after_forward()
                    loss = self.loss_fn(out, y)
                
                if is_training:
                    opt.zero_grad()
                    cb.before_backward()
                    loss.backward()
                    opt.step()
                
                phase.batch_loss = loss.item()
                cb.batch_ended(phase=phase, output=out, target=y)
                
            cb.phase_ended(phase=phase)
                    
        cb.epoch_ended(phases=phases, epoch=curr_epoch)

In [ ]:
#export
class TrainingInterrupted(Exception):
    def __init__(self, context=None):
        self.context = context

In [ ]:
#export
def place_and_unwrap(batch, device):
    """Places tensors from batch onto proper device and converts targets
    into proper shape depending on number of tensors.
    """
    batch = [t.to(device) for t in batch]
    x, *ys = batch
    return x, unwrap_if_single(ys)

In [ ]:
def get_mnist():
    from torchvision.datasets import MNIST
    from torchvision import transforms as T
    
    root = defaults.datasets/'mnist'

    mnist_stats = ([0.15]*1, [0.15]*1)

    trn_ds = MNIST(root, train=True, transform=T.Compose([
        T.Resize(32),
        T.RandomAffine(5, translate=(0.05, 0.05), scale=(0.9, 1.1)),
        T.ToTensor(),
        T.Normalize(*mnist_stats)
    ]))
    val_ds = MNIST(root, train=False, transform=T.Compose([
        T.Resize(32),
        T.ToTensor(),
        T.Normalize(*mnist_stats)
    ]))
    
    return trn_ds, val_ds

In [ ]:
from torch.nn import functional as F

In [ ]:
loop = Loop(TinyNet(1), cbs=[C.Average(accuracy)], loss_fn=F.cross_entropy)

In [ ]:
loop.fit_datasets(*get_mnist(), epochs=3, batch_size=2048)

In [ ]:
loop.cb['history'].plot();